In [ ]:
import gspread
import pandas as pd
import openai
import smtplib
import matplotlib.pyplot as plt
from oauth2client.service_account import ServiceAccountCredentials
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [ ]:
# Google Sheets Authentication
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("google_sheets_credentials.json", scope)
client = gspread.authorize(creds)

In [ ]:
sheet = client.open("Personal Finance Tracker").sheet1  
data = sheet.get_all_records()

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
description_col = "Notes" \

def categorize_expense(description):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an AI that categorizes expenses."},
                {"role": "user", "content": f"Categorize this expense: {description}"}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error in API call: {e}")
        return "Uncategorized"  

In [ ]:
with open("openai_key.txt", "r") as file:
    api_key = file.read().strip()  


import openai
client = api_key

In [ ]:
spending_summary = df.groupby("Category")["Amount"].sum().to_string()

# AI prompt for financial insights
budget_insight_prompt = f"Analyze the following spending data and provide budgeting tips:\n{spending_summary}"

# ** Fixed API Call **
response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": budget_insight_prompt}]
)

# Extract response
budget_insights = response.choices[0].message.content
print("\n💰 Budgeting Insights from AI:\n", budget_insights)

In [ ]:
df = df.loc[df['Type']=='Expense']

In [ ]:
df['Amount'] = pd.to_numeric(df['Amount'])

In [ ]:
# Generate Expense Report
plt.figure(figsize=(8, 5))
df.groupby("Category")["Amount"].sum().plot(kind="bar", color="skyblue")
plt.title("Spending Breakdown")
plt.ylabel("Amount Spent ($)")
plt.savefig("spending_report.png")

In [ ]:
# Send Email with Summary
def send_email():
    sender_email = "your_email@gmail.com"
    receiver_email = "recipient_email@gmail.com"
    password = "your_app_password"

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = "📊 Monthly Finance Report"

    body = f"Here is your monthly spending summary:\n\n{spending_summary}\n\nBudgeting Insights:\n{budget_insights}"
    msg.attach(MIMEText(body, "plain"))

    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

send_email()

print("✅ AI Finance Report Sent!")